# Прогнозирование цен потребительского ритейла по тестовой выборке на основе глубоких нейронных сетей
__Выполнил:__ *Домченко Максим*

__Студент группы:__ *РИМ-130962*

In [ ]:
# Для локального запуска загрузите kaggle.json в ~/.kaggle/kaggle.json и дайте права:
# !chmod 600 ~/.kaggle/kaggle.json

In [16]:
# Для запуска в colab

# from google.colab import files
# uploaded = files.upload('/root/.kaggle')
# !chmod 600 /root/.kaggle/kaggle.json

In [14]:
!kaggle competitions download -p ../content -c favorita-grocery-sales-forecasting --force

 61%|████████████████████████▍               | 280M/458M [00:00<00:00, 2.93GB/s]
100%|████████████████████████████████████████| 458M/458M [00:00<00:00, 2.92GB/s]
